## Install packages

In [39]:
# !pip install numpy
# !pip install IPython
# !pip install midiutil
# !pip install midi2audio
# !pip install music21
# !pip install pygame
# !pip install scikit-learn
# !wget https://raw.githubusercontent.com/musescore/MuseScore/master/share/sound/FluidR3Mono_GM.sf3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 12.8 MB/s eta 0:00:0000:0100:01


In [19]:
# Import after installing

import numpy as np
import IPython
from IPython.display import Audio, display
import midiutil
import midi2audio
import music21

## Define the frequencies of notes

In [20]:
NOTE_FREQUENCIES = {
   'C4': 261.63, 'D4': 293.66, 'E4': 329.63, 'F4': 349.23,
   'G4': 392.00, 'A4': 440.00, 'B4': 493.88, 'C5': 523.25,
   'F3': 174.61, 'G3': 196.00, 'A3': 220.00, 'B3': 246.94
}

## Make a single note sine wave and play

In [21]:
def create_sine_wave(frequency, duration, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration))
    wave = np.sin(2 * np.pi * frequency * t)
    return wave

In [22]:
single_note = create_sine_wave(NOTE_FREQUENCIES['A4'], 1)
#440Hz, 1 second
display(Audio(single_note, rate=44100))

## Make a single note square wave and play

A square wave is made up of a fundamental frequency plus many odd-numbered harmonics (3x, 5x, etc.)
In this example, we will use harmonics 3x, 5x, ..., 19x, based on the following equation.

$\text{square}(f, t) = \frac{4}{\pi} \sum_{k=1,3,5,...,19}^{} \frac{1}{k} \sin(2\pi k f t)$

In [23]:
def create_square_wave(frequency, duration, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration))
    wave = np.zeros_like(t)
    for k in range(1, 21, 2):
        wave += (1 / k) * np.sin(2 * np.pi * k * frequency * t)
    wave *= 4 / np.pi
    return wave

In [24]:
single_note = create_square_wave(NOTE_FREQUENCIES['A4'], 1)
#440Hz, 1 second
display(Audio(single_note, rate=44100))

## Make a chord and play

In [ ]:
def create_chord(frequencies, duration, sample_rate=44100):
    waves = [create_sine_wave(f, duration, sample_rate) for f in frequencies]
    return np.mean(waves, axis=0)

chord_frequencies = [NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['G4']] #C major chord
chord = create_chord(chord_frequencies, 2)

In [ ]:
chord_progression = [
       ([NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['G4']], 2.0),#C Major chord for 2 secs
       ([NOTE_FREQUENCIES['F3'], NOTE_FREQUENCIES['A3'], NOTE_FREQUENCIES['C4']], 1.0),#F Major chord for 1 sec
       ([NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['G4']], 1.0),#C Major Chord for 1 sec
       ([NOTE_FREQUENCIES['F3'], NOTE_FREQUENCIES['A3'], NOTE_FREQUENCIES['C4']], 1.0),#F Major Chord for 1 sec
       ([NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['G4']], 1.0),#C Major Chord for 1 sec
       ([NOTE_FREQUENCIES['G3'], NOTE_FREQUENCIES['B3'], NOTE_FREQUENCIES['D4']], 1.0),#G Major Chord for 1 sec
       ([NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['G4']], 1.0)]#C Major Chord for 1 sec
chord_waves = []
for frequencies, duration in chord_progression:
    chord_wave = create_chord(frequencies, duration)
    chord_waves.append(chord_wave)
backing_track = np.concatenate(chord_waves)
display(Audio(backing_track, rate=44100))

## Make a melody line and play

In [ ]:
from IPython.display import Image, display
display(Image(url="https://musescore.com/static/musescore/scoredata/g/d08471e13f9aa1a2b2605bb7347943654b012f0e/score_0.svg?no-cache=1715697879"))

In [ ]:
melody_notes = [
       NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['C4'], NOTE_FREQUENCIES['G4'], NOTE_FREQUENCIES['G4'],
       NOTE_FREQUENCIES['A4'], NOTE_FREQUENCIES['A4'], NOTE_FREQUENCIES['G4'],
       NOTE_FREQUENCIES['F4'], NOTE_FREQUENCIES['F4'], NOTE_FREQUENCIES['E4'], NOTE_FREQUENCIES['E4'],
       NOTE_FREQUENCIES['D4'], NOTE_FREQUENCIES['D4'], NOTE_FREQUENCIES['C4']
   ]
melody_durations = [0.5] * 14 #14 notes (0.5 sec for one beat)
melody_durations[6] = 1.0#The 7th note lasts for two beats
melody_durations[-1] = 1.0#The last note lasts for two beats

In [ ]:
def create_melody(notes, durations, sample_rate=44100):
   waves = []
   for note, duration in zip(notes, durations):
       wave = create_sine_wave(note, duration, sample_rate)#Generate a sine wave for each note
       waves.append(wave)#Append to the list
   return np.concatenate(waves)#Concatenate

melody = create_melody(melody_notes, melody_durations)
display(Audio(melody, rate=44100))

## Adjust amplitudes, mix and play

In [ ]:
melody = melody * 0.7 #adjust the amplitude
backing_track = backing_track * 0.3#adjust the amplitude
combined = melody + backing_track#combine
combined = combined / np.max(np.abs(combined))
display(Audio(combined, rate=44100))

## Write a MIDI file (the initial melody line from Twinkle Twinkle Little Star) and save

In [25]:
from midiutil import MIDIFile # Import library

midi = MIDIFile(1) # Create a MIDI file that consists of 1 track
track = 0 # Set track number
time = 0 # Where is the event placed (at the beginning)
tempo = 120 # The tempo (beats per minute)
midi.addTempo(track, time, tempo) # Add tempo information

In [26]:
help(midi.addTempo)

Help on method addTempo in module midiutil.MidiFile:

addTempo(track, time, tempo) method of midiutil.MidiFile.MIDIFile instance
    Add notes to the MIDIFile object
    
    :param track: The track to which the tempo event  is added. Note that
        in a format 1 file this parameter is ignored and the tempo is
        written to the tempo track
    :param time: The time (in beats) at which tempo event is placed
    :param tempo: The tempo, in Beats per Minute. [Integer]



In [27]:
notes = [60, 60, 67, 67, 69, 69, 67, # CCGGAAG
         65, 65, 64, 64, 62, 62, 60] # FFEEDDA
durations = [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2]

In [28]:
current_time = 0
for pitch, duration in zip(notes, durations):
   midi.addNote(track, 0, pitch, current_time, duration, 100)
   current_time += duration
   print(pitch, duration)

# add notes to MIDI file
with open("twinkle.mid", "wb") as f:
   midi.writeFile(f) # write MIDI file

60 1
60 1
67 1
67 1
69 1
69 1
67 2
65 1
65 1
64 1
64 1
62 1
62 1
60 2


## Convert the MIDI file into an audio file using downloaded synthesizer

In [29]:
from midi2audio import FluidSynth # Import library
fs = FluidSynth("FluidR3Mono_GM.sf3") # Initialize FluidSynth

In [30]:
# Have to add fluidsynth to your system path for this to work...
# See e.g. https://stackoverflow.com/questions/75353745/how-do-i-set-up-midi2audio-on-windows
# I had to follow the suggested steps to get this to work on my tablet

In [31]:
fs.play_midi("twinkle.mid")

In [32]:
fs.midi_to_audio("twinkle.mid", "twinkle.wav") # convert MIDI to audio

In [33]:
display(Audio('twinkle.wav'))

## Write a MusicXML file and save

In [34]:
from music21 import *
melody = stream.Stream()
melody.append(metadata.Metadata())
melody.metadata.title = 'Twinkle Twinkle Little Star'
melody.metadata.composer = 'Unknown'
melody.append(meter.TimeSignature('4/4'))
melody.append(key.Key('C'))

In [35]:
notes = [
   ('C4', 'quarter'),
   ('C4', 'quarter'),
   ('G4', 'quarter'),
   ('G4', 'quarter'),
   ('A4', 'quarter'),
   ('A4', 'quarter'),
   ('G4', 'half'),   
]
for pitch, duration in notes:
   n = note.Note(pitch)
   n.duration.type = duration
   melody.append(n)

In [36]:
melody.write('musicxml', fp='twinkle.musicxml')

PosixPath('/Users/haven/workspace/cse153_workbooks/final_hw1/workbook/twinkle.musicxml')

## Generate and receive midi events using pygame

In [40]:
import pygame
import pygame.midi
from pygame.locals import *
import random
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [41]:
pygame.init()
pygame.fastevent.init()
pygame.midi.init()

In [42]:
# This will (probably) receive input from an external controller,
# and output to your machine. By changing the ids you could also
# output to a controller. You can delete the parts relating to inputs
# if you don't have a midi controller
input_id = pygame.midi.get_default_input_id()
output_id = pygame.midi.get_default_output_id()

In [43]:
input_id, output_id

(-1, -1)

In [44]:
i = pygame.midi.Input(input_id)
player = pygame.midi.Output(output_id)
player.set_instrument(0)

MidiException: 'Device id is -1, not a valid output id.  -1 usually means there were no default Output devices.'

In [29]:
# Play a random note
r = random.choice(range(30,90)) # Pitch range
time.sleep(0.25)
player.note_on(r, 50)
time.sleep(0.25)
player.note_off(r)

In [30]:
going = False
#going = True # Uncomment if you want to run this part...

clock = pygame.time.Clock()

while going:
    clock.tick(60)
    if i.poll():
        midi_events = i.read(10)
        # print(str(midi_events)) # dump all the info...
        for e in midi_events:
            note_info, t = e
            z,note,velocity,_ = note_info
            if velocity > 0:
                print((note, velocity))
                player.note_on(note, velocity)
            if velocity == 0:
                player.note_off(note)

# Classifiying Song's Emotions from BPM and Keys Using Scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#List of songs on Billboard's 2024 Year-End Hot 100 chart (from No.1 to No.10)
song_titles = ["Lose Control", "A Bar Song (Tipsy)", "Beautiful Things", "I Had Some Help", "Lovin on Me", "Not Like Us", "Espresso", "Million Dollar Baby", "I Remember Everything", "Too Sweet"]
# For example, the BPM of "Lose Control" is 128.
BPMs = [128, 81, 105, 126, 105, 101, 104, 138, 78, 117]
# For example, the key of "Lose Control" is minor, but the key of "Beautiful Things" is major.
is_major =  [0, 0, 1, 1, 1, 1, 1, 0, 1, 0]
# For example, the emotion of "Lose Control" is negative, but the emotion of "A Bar Song (Tipsy)" is positive
is_positive = [0, 1, 1, 0, 1, 0, 1, 1, 0, 1]

In [14]:
#Stacking inputs
X = np.column_stack((np.array(BPMs), np.array(is_major)))
#Splitting the dataset (8:2 ratio)
X_train, X_test, Y_train, Y_test = train_test_split(X, is_positive, test_size=0.2, random_state=42)
#Train your model
model = LogisticRegression(random_state=42)
model.fit(X_train, Y_train)

LogisticRegression(random_state=42)

In [17]:
#Test your model and make a report
y_pred = model.predict(X_test)
my_report = classification_report(Y_test, y_pred)
my_report

/Users/haven/opt/anaconda3/envs/feiyueh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haven/opt/anaconda3/envs/feiyueh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haven/opt/anaconda3/envs/feiyueh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         1\n           1       0.50      1.00      0.67         1\n\n    accuracy                           0.50         2\n   macro avg       0.25      0.50      0.33         2\nweighted avg       0.25      0.50      0.33         2\n'